In [ ]:
"""
重點流程：

- 掛載google drive
- 安裝套件
- 引入套件
- 建置主程序(含line_bot_api跟 webhookhandler)
- 建置主程序的API接口 (這樣才能接收用戶訊息)

* 先設定好當收到用戶訊息時，機器人要回傳給用戶什麼東西(如回覆給用戶文字，或是回覆圖片)，建立一個字典檔
* 把用戶的訊息丟回上步驟已經設定好的字典檔裡面看有沒有可以回傳的值

- 最後兩個步驟去做調整修改 

"""



In [ ]:
"""
資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

"""

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
安裝套件
"""
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
"""
引用套件
"""


# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import LineBotApi, WebhookHandler

# 引用無效簽章錯誤
from linebot.exceptions import InvalidSignatureError

In [ ]:
'''
建置主程序app

建置handler與 line_bot_api
'''

# 設定Server啟用細節
# 準備app這個flask HTTP Application，並請ngrok給我們這個app一個臨時的HTTP URL
app = Flask(__name__, static_url_path="/material", static_folder="./material/")
run_with_ngrok(app)



# 生成實體物件
# 這是跟LINE溝通用 (設置公司業務)
line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")

# 收發消息用 (設置前台小秘書，負責接收並過濾消息用)
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")


In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
# 讓line丟消息過來的地方
@app.route("/", methods=["POST"])
def callback():
    # 取得簽章，可以增加不能減少
    signature = request.headers["X-Line-Signature"]

    # 取得用戶消息 (習慣會打印出來看一下)
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log (把用戶的消息記錄起來，並保存在GOOGLE DRIVE上)
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # 把消息交給handler做處理
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return "OK"

In [ ]:
"""

消息素材準備

當用戶有文字訊息傳入時，要回覆的的文字訊息

"""

# 從linebot.models引入訊息模型，文字收取與文字寄發訊息，及影像訊息
from linebot.models import MessageEvent, TextMessage, TextSendMessage, ImageMessage, ImageSendMessage

# xxxxSendMessage，回應用戶的訊息物件(回應文字訊息)
specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")


# 這則是回應用戶的圖片消息，參數跟line_bot_designer很類似
# 注意，圖片必須是HTTP網址
# 未來可以把照片保存到Cloud Storage
analyze_truth_image_message = ImageSendMessage(
      original_content_url="https://i.imgur.com/M8jnn9B.png",
      preview_image_url="https://i.imgur.com/LtCtgg0.jpg"
      )

In [ ]:
"""

設定一個字典，把用戶傳進來的值當作字典的key，並回傳value
未來可以是資料庫，把你要回應的訊息存在資料庫內，當用戶傳訊息時，會從資料庫裡面找資料回傳給用戶

"""

template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message
}

# 如上述，當用戶在客戶端輸入@123時會回傳上一個cell裡面的specific_text_messaage ; 輸入@分析的本質，同理



In [ ]:
"""

對用戶送來的訊息做出判斷
並給予回應

"""


# 用戶發出文字消息時， 按條件內容, 回傳文字消息
# @handler.add(Event)
#　MessageEvent => 消息事件
# 文字消息(TextMessage)、圖片消息(ImageMessage)、影片消息(VideoMessage)等等...

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # 當用戶傳來的消息有@符號時，我們就從字典中找回應 reply_message(event.reply_token, [TextSendMessage(回應給用戶的文字)])
    if (event.message.text.find("@") != -1):
        line_bot_api.reply_message(event.reply_token, template_message_dict.get(event.message.text))
        
    # 若找不到結果，則傳回文字消息，查無此字
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質")
        )

# TextMessage   =>  用戶傳來的訊息
# TextSendMessage =>  LINE傳給用戶的訊息


In [ ]:
# 主程序運行
app.run()